## validation set

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# train data에서 validation set 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
# train
dt.fit(sub_input, sub_target)

# score
print(dt.score(sub_input, sub_target))
# test set이 아닌 validation set으로 평가
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## cross validation

In [ ]:
from sklearn.model_selection import cross_validate
# cross_validate() 함수는 기본적으로 5-fold cross validation을 수행
# cv 매개변수 값으로 fold 수 지정 가능
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01016903, 0.0116148 , 0.01081276, 0.01055217, 0.01019025]), 'score_time': array([0.00277519, 0.00162005, 0.00159836, 0.00161958, 0.00267005]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
# 평균 계산
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

# cv 매개변수로 splitter 지정
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

In [ ]:
# splitter 클래스의 n_splits 매개변수 값으로 fold 개수(k) 지정
# 10-fold cross validation
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

## 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

# 매개변수와 후보값들의 리스트를 dictionary로 만들기
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# params를 전달하며 Grid Search 객체 생성
# n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수 지정: -1로 지정하면 모든 코어를 사용
# cv 매개변수 기본값은 5 (5-fold cross validation)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# params 값마다 5-fold cross validation 수행
gs.fit(train_input, train_target)
# 검증이 끝나면 score가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 모델 훈련하고 gs.best_estimator_ 속성에 저장

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
# 저장된 모델의 score
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# best_params_ 속성에 찾은 최적의 매개변수 저장됨
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# 넘파이 arange()나 파이썬 range()를 이용해 배열을 만들어 매개변수로 줄 수 있음
# params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
#           'max_depth': range(5, 20, 1),
#           # min_samples_split: 노드를 나누는 최소 샘플 수
#           'min_samples_split': range(2, 100, 10)
#           }

### 랜덤 서치

In [ ]:
# 확률 분포 클래스
from scipy.stats import uniform, randint

In [ ]:
# 0~10 사이의(0~9까지 포함) 범위
rgen = randint(0, 10)
# 10개의 정수 샘플링
rgen.rvs(10)

array([9, 1, 8, 5, 4, 7, 2, 6, 9, 1])

In [ ]:
# 1000개 샘플링하여 각 정수값의 개수 세기
np.unique(rgen.rvs(1000), return_counts=True)

# 결과를 확인하면 어느정도 균등하게 샘플링됨

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106,  96, 106,  98,  93,  88, 119,  87, 104, 103]))

In [ ]:
# 0~1 사이의 범위
ugen = uniform(0, 1)
# 10개의 실수 샘플링
ugen.rvs(10)

array([0.76125445, 0.29597466, 0.00650326, 0.70617209, 0.69318374,
       0.64129573, 0.72857217, 0.25058781, 0.27633346, 0.76252958])

In [ ]:
# 매개변수
# 탐색할 매개변수 범위를 지정
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          # min_samples_leaf: 리프 노드의 최소 샘플 개수(분할하여 만들어질 자식 노드의 최소 샘플 개수) - 이 값보다 작으면 분할하지 않음
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
# 랜덤 서치 클래스
from sklearn.model_selection import RandomizedSearchCV

# params를 전달하며 랜덤 서치 객체 생성
# n_iter 값으로 샘플링 횟수 지정 - 매개변수 범위에서 100번 샘플링하여 교차 검증 수행
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e0e64f66a10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e0e64fb2b90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e0e64fb2f50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e0e64fb3730>},
                   random_state=42)

In [ ]:
  dt = gs.best_estimator_
  # 저장된 모델의 score
  print(dt.score(train_input, train_target))
  print(dt.score(test_input, test_target))

0.8928227823744468
0.86


In [ ]:
# best_params_ 속성에 찾은 최적의 매개변수 저장됨
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
